In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Set your OpenAI API key
load_dotenv()
api_key = os.getenv("API_KEY")

# Initialize the OpenAI API client
client = OpenAI(api_key=api_key)

def generate_response(prompt):
    """
    Generates a response from the OpenAI API based on the given prompt.

    Args:
        prompt (str): The prompt/query to send to the API.

    Returns:
        str: The API response.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt},
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

# 1- User writes a One-Paragraph Summary

In [ ]:
summary = ""

# 2- Character Summaries: Create brief character descriptions.

In [ ]:
prompt = """
Imagine a story with this summary: '{{summary}}'. I need you to create brief character descriptions for this story. Feel free to add more characters or modify the prompts as needed. Please write a dict of characters in the following formats, without any other text:
{{
    "characterName": {{
        "Name": "",
        "Role": "",
        "Age": "",
        "Occupation": "",
        "Appearance": "",
        "Personality Traits": "",
        "Motivation": "",
        "Backstory": ""
    }}
}}
"""

## WE COULD HAVE CHARACTER ROLES HERE: ANTAGONISTS; PROTAGONISTS; ETC.

characters = generate_response(prompt)

## THIS COULD RUN 2 TIMES; SO WE HAVE EXTRA CHARACTERS.

# 3- Character Charts: Develop detailed character profiles.

In [ ]:
for character in characters:
    prompt = """
        Consider the story '{{summary}}'. I need you to write a detailed character profile for this character '{{character}}' who will appear in it. Also, list a few key scenes where you think they can play a significant role in the story. Please answer with the format:
        {
            'characterProfile': 'Your Answer',
            'keyScenes': 'Your Answer'
        }
    """

    response = generate_response(prompt)
    character["detailedProfile"] = response["characterProfile"]
    character["keyScenes"] = response["keyScenes"]


# 4- Plot Summary: Outline the major plot points.

In [ ]:
prompt = """
    Brainstorm and write the major plot points for a story that has this summary '{summary}', and involves these characters: {characters}
"""

plotPoints = generate_response(prompt)

## Maybe it should return the plot points in some format?

# 5- Scene List: Create a list of scenes in the novel.

In [ ]:
prompt = """
    Considering this logline '{summary}', these major plot points '{plot_points}', and these characters '{characters}', write a dictionary of scenes for a novel of scenes for a novel, describing what happens in the scene, who participates in it, what are their goals and what is the end result of the scene. Use scene numbers as keys and scene details as values. 
"""

sceneLogLines = generate_response(prompt)

# 6- Expand Each Scene: Write a paragraph for each scene.

In [ ]:
scenes = []

for logLine in sceneLogLines:
    prompt = """
 'Considering the {scene}', brainstorm the most creative output of description for the scene, with the participation of '{characters}'. You can create other characters and add them, make up situations that you seem plausible AND innovative, and mark any possible plot twist with <**PLOT TWIST**> markers. Please consider the character goals, and the end result. Answer everything using this format: 
scene = {
    'description': "anything you want the scene description to be",
    'characters': ['a list of character names that participate in it'],
    'plot_twist': "the plot twists you find interesting to add here"
}
"""
    scenes.append(generate_response(prompt))

# 7- Character Synopses: Write a one-page synopsis for each major character.

In [ ]:
for character in characters:
    # Filter scenes involving the target character
    filtered_scenes = [scene for scene in scenes if character in scene['characters']]    

    prompt = """
        Considering this character {character}, that appears in these scenes {filtered_scenes} of the novel with this summary {summary} and write a long character synopsis of said character role in the novel.
    """

    character["synopsis"] = generate_response(prompt)

# 8- Synopsis: Expand the plot summary into a multi-page synopsis.

In [ ]:
part_size = len(scenes) // 3

part1 = scenes[:part_size]
part2 = scenes[part_size:2 * part_size]
part3 = scenes[2 * part_size:]

parts = [part1, part2, part3]

acts = []

for part in parts:
    prompt = """
        Considering the summary of these scenes: {part}, write the synopsis of this act of the novel.
    """

    acts.append(generate_response(prompt))


# 9- Write the Novel: Begin writing your novel based on the detailed synopsis.

In [ ]:
chapters = []

for scene in scenes:
    prompt = """
        Considering the summary of this scene {scene}, which belongs to this act of the story {act}, and the characters {characters}, write the scene.
    """

    chapters.append(generate_response(prompt))

# 10- Join all the chapters into a txt and export it.

In [ ]:
def write_chapters_to_txt(chapters, output_file):
    """
    Writes the chapters (list of texts) to a text file.

    Args:
        chapters (list): List of text chapters.
        output_file (str): Path to the output text file.
    """
    try:
        with open(output_file, 'w') as file:
            for chapter in chapters:
                file.write(chapter + '\n')
        print(f"Chapters successfully written to {output_file}")
    except Exception as e:
        print(f"Error writing chapters to {output_file}: {e}")

output_file = "book.txt"
write_chapters_to_txt(chapters, output_file)